In [1]:
import numpy as np
import pandas as pd
np.e

2.718281828459045

In [2]:
A = .0001234 
B = .000004
c = 1.122

In [3]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [4]:
class makehams:
    def __init__(self, a, b, c, l_0 = 1_000_000, w = 120):
        self.a = a
        self.b = b
        self.c = c
        self.l_0 = l_0
        self.w = w
        
    def mu (self, x): 
        mu_x = self.a + self.b * self.c ** x
        return mu_x
        
    def S (self, x):
        a = self.a; b = self.b; c = self.c
        s_x = np.e ** ( - (a * x) - ( b * (c ** x) - b ) / np.log(c) )
        return s_x
                   
    def q (self, x, t, u):
        tbaru_q_x = ( self.S(x + t) - self.S(x + t + u) ) / self.S(x)
        if x + t + u >= 121:
            tbaru_q_x = 1
        return tbaru_q_x
        
    def p (self, x,t):
        a = self.a; b = self.b; c = self.c
        t_p_x = np.e ** ( -a * t - b/np.log(c) * c ** x * (c ** t - 1) ) 
        return t_p_x
        
    def l (self, x, t):
        l_x = self.l_0 * p(self, x, t)
        return l_x
        
    def t_p_x_list (self, x):
        """return list of consecutive t_p_x values"""
        w = self.w
        next_age = int(x + 1)
        tps = {}
        # ps[x] = self.p(x, next_age-x)
        
        for t in range (0, w - next_age + 1):
            tps[f'{np.round(t + (next_age - x),2)}_p_{x}'] = self.p(x, t + (next_age - x))
            
        tps[f'{np.round(w - x + 1,2)}_p_{x}'] = 0
        return tps

    def p_x_list (self, x):
        w = self.w
        next_age = int(x + 1)
        ps = {}
        ps[f'p_{x}'] = self.p(x, next_age - x)
        for t in range (0, w - next_age):
            ps[f'p_{next_age + t}'] = self.p(next_age + t, 1)
        ps[f'p_{w}'] = 0
        return ps
        
    def mu_x_list(self,x):
        return {f'mu_{x + t}': self.mu(x+t) for t in range(0,self.w - x + 1)}
        
    def q_x_list(self, x):
        return {f'q_{x + t}': self.q(x + t, 0, 1) for 
                t in range(0, self.w - x + 1)}
        
    def l_x_list(self,x):
        return dict(zip(
            [f'l_{x+t}' for t in range(0, self.w - x + 1)],
            self.l_0*np.array([1] + list(self.t_p_x_list(x).values())[:-1])
        ))
        
    def e_x_list(self, x):
        w = self.w
        next_age = int(x + 1)
        curt_life_list = {}
        curt_life_list[f'e_{x}'] = sum(list(self.t_p_x_list(x).values()))
        for t in range(0, w - next_age + 1):                                    
            curt_life_list[f'e_{next_age+t}'] = \
                                        sum(list(
                                        self.t_p_x_list(next_age+t).values()))
        return curt_life_list

    def stats_table(self, x):
        table = pd.DataFrame({
            'mu_x': self.mu_x_list(x).values(),
            'p_x': self.p_x_list(x).values(),
            'q_x': self.q_x_list(x).values(),
            'l_x': self.l_x_list(x).values(),
            'e_x': self.e_x_list(x).values()
        }, index = list(range(x, self.w + 1))).round(4)
        table.index.name = 'age'
        return table
mortality_model = makehams(A, B, c, 1_000_000)

In [5]:
mortality_model.stats_table(20).to_csv('./hw_2_python_soln.csv')